In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install rake-nltk

In [ ]:
import flask
import werkzeug
import keras.models
import numpy
import scipy.misc
import imageio
import cv2
from flask_ngrok import run_with_ngrok
import numpy as np
import pandas as pd
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict


app = flask.Flask(__name__)
run_with_ngrok(app)


@app.route('/', methods=['GET', 'POST'])
def handle_request():
    movie = flask.request.args.get('movie')
    rec_dict = defaultdict(list)
    df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

    df = df[['Title','Genre','Director','Actors','Plot']]
    # discarding the commas between the actors' full names and getting only the first three names
    df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])

    # putting the genres in a list of words
    df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

    df['Director'] = df['Director'].map(lambda x: x.split(' '))

    # merging together first and last name for each actor and director, so it's considered as one word 
    # and there is no mix up between people sharing a first name
    for index, row in df.iterrows():
        row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
        row['Director'] = ''.join(row['Director']).lower()
    # initializing the new column
    df['Key_words'] = ""

    for index, row in df.iterrows():
        plot = row['Plot']
    
        # instantiating Rake, by default is uses english stopwords from NLTK
        # and discard all puntuation characters
        r = Rake()

        # extracting the words by passing the text
        r.extract_keywords_from_text(plot)

        # getting the dictionary whith key words and their scores
        key_words_dict_scores = r.get_word_degrees()
    
        # assigning the key words to the new column
        row['Key_words'] = list(key_words_dict_scores.keys())
    # dropping the Plot column
    df.drop(columns = ['Plot'], inplace = True)
    df.set_index('Title', inplace = True)
    df['bag_of_words'] = ''
    columns = df.columns
    for index, row in df.iterrows():
        words = ''
        for col in columns:
            if col != 'Director':
                words = words + ' '.join(row[col])+ ' '
            else:
                words = words + row[col]+ ' '
        row['bag_of_words'] = words
    
    df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)
    count = CountVectorizer()
    count_matrix = count.fit_transform(df['bag_of_words'])

    # creating a Series for the movie titles so they are associated to an ordered numerical
    # list I will use later to match the indexes
    indices = pd.Series(df.index)
    indices[:5]
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    recommended_movies = []
    rec = ""
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == movie].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
    for s in recommended_movies:
        rec = rec+"^"+s
    rec = rec[1:]

    return rec
    
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b2166ac6caad.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Jun/2021 07:35:36] "GET /?movie=The%20Lion%20King HTTP/1.1" 200 -
